In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.interpolate import griddata

# Load the HYCOM dataset
hycom_ds = xr.open_dataset('output.nc')
scale_factor = hycom_ds['surf_el'].attrs.get('scale_factor', 1.0)
hycom_surf_el = hycom_ds['surf_el'] * scale_factor
hycom_lon = hycom_ds['lon'].values
hycom_lat = hycom_ds['lat'].values

# Adjust HYCOM longitudes
hycom_lon = np.where(hycom_lon > 180, hycom_lon - 360, hycom_lon)
hycom_surf_el_cleaned = hycom_surf_el.where(hycom_surf_el != hycom_ds['surf_el'].attrs.get('_FillValue', np.nan))

# Load the ADCIRC dataset
adcirc_ds = xr.open_dataset('/scratch/07174/soelem/global_2-20km/fort.63.nc')
adcirc_lon = adcirc_ds['x'].values
adcirc_lat = adcirc_ds['y'].values
adcirc_fill_value = adcirc_ds['zeta'].attrs.get('_FillValue', np.nan)

# Start ADCIRC from the 360th timestep, selecting every 3rd timestep
adcirc_start_index = 360
adcirc_time_sel = adcirc_ds.isel(time=slice(adcirc_start_index, None, 3))

# Initialize RMSE storage array
squared_diff_sum = np.zeros((len(hycom_lat), len(hycom_lon)))
min_timesteps = min(len(hycom_ds['time']), len(adcirc_time_sel['time']))

for t in range(min_timesteps):
    hycom_surf_el_t = hycom_surf_el_cleaned.isel(time=t).values
    adcirc_zeta_t = adcirc_time_sel['zeta'].isel(time=t).where(adcirc_time_sel['zeta'] != adcirc_fill_value).values

    # Check if ADCIRC zeta is 2D (assume 1st dimension is the spatial size)
    if adcirc_zeta_t.ndim == 2:
        adcirc_zeta_t_flat = adcirc_zeta_t.ravel()  # Flatten the 2D ADCIRC data

        # Apply the same to coordinates, to match the spatial dimension
        adcirc_lon_flat = np.tile(adcirc_lon, (adcirc_zeta_t.shape[1], 1)).T.ravel()  # Match to zeta's shape
        adcirc_lat_flat = np.tile(adcirc_lat, (adcirc_zeta_t.shape[1], 1)).T.ravel()  # Match to zeta's shape
    else:
        adcirc_zeta_t_flat = adcirc_zeta_t.ravel()
        adcirc_lon_flat = adcirc_lon
        adcirc_lat_flat = adcirc_lat

    # Remove NaNs from ADCIRC zeta and corresponding coordinates
    valid_mask = ~np.isnan(adcirc_zeta_t_flat)
    adcirc_lon_valid = adcirc_lon_flat[valid_mask]
    adcirc_lat_valid = adcirc_lat_flat[valid_mask]
    adcirc_zeta_valid = adcirc_zeta_t_flat[valid_mask]

    # Interpolate ADCIRC data onto the HYCOM grid
    adcirc_interpolated_t = griddata(
        (adcirc_lon_valid, adcirc_lat_valid),
        adcirc_zeta_valid,
        (hycom_lon[None, :], hycom_lat[:, None]),
        method='linear'
    )

    # Handle any NaN values in the interpolated ADCIRC data
    adcirc_interpolated_t = np.ma.masked_invalid(adcirc_interpolated_t)

    # Compute squared difference between HYCOM and interpolated ADCIRC
    squared_diff = (hycom_surf_el_t - adcirc_interpolated_t) ** 2
    squared_diff_sum += np.nan_to_num(squared_diff)

# Compute RMSE by averaging squared differences and taking the square root
rmse = np.sqrt(squared_diff_sum / min_timesteps)

# Plot the RMSE
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
contour = ax.contourf(hycom_lon, hycom_lat, rmse, 60, cmap='Reds', transform=ccrs.PlateCarree())
plt.colorbar(contour, label='RMSE in Water Surface Elevation (m)')
ax.coastlines()

land = cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='face', facecolor='darkgray')
ax.add_feature(land)
plt.title('RMSE between HYCOM and ADCIRC Water Surface Elevation (Aug 2024)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()